<a href="https://colab.research.google.com/github/MODIKULDEEP/BART-Dialogue-Summarization/blob/main/BART_Dialogue_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate transformers[torch]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
"""
BART HAS 400M PARAMS: https://github.com/facebookresearch/fairseq/tree/main/examples/bart
"""
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
!pip install py7zr

In [ ]:
from datasets import load_dataset
dataset = load_dataset("samsum")
dataset

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
sample = dataset['test'][0] ['dialogue']
label = dataset['test'][0]['summary']
def generate_summary(input, llm):
  input_prompt = f"""
                  Summarize the following conversation.

                  {input }

                  Summary :
                  """

  input_ids = tokenizer(input, return_tensors='pt')
  tokenized_output = llm.generate(input_ids['input_ids' ], min_length=30, max_length=200)
  output = tokenizer.decode(tokenized_output[0], skip_special_tokens=True)

  return output

output = generate_summary(sample, llm=model)
print("Sample")
print(sample)
print ("------------------------")
print("Model Generated Summary:")
print (output)
print("Correct Summary:")
print (label)

Sample
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
------------------------
Model Generated Summary:
Hannah: Hey, do you have Betty's number? Amanda: Lemme check. Hannah: Ask Larry. Amanda: He called her last time we were at the park together.
Correct Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


In [ ]:
def tokenize_inputs(example):
  start_prompt = "Summarize the following conversation.\n\n"
  end_prompt = "\n\nSummary: "
  prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
  example['input_ids'] = tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt', max_length=512).input_ids
  example['labels'] = tokenizer(example['summary'], padding='max_length', truncation=True, return_tensors='pt', max_length=512).input_ids

  return example

tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = dataset.map(tokenize_inputs, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'dialogue', 'summary'])
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Filter:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets['train'].shape)
print(tokenized_datasets['validation'].shape)
print(tokenized_datasets['test'].shape)

(148, 2)
(9, 2)
(9, 2)


In [ ]:
tokenized_datasets['train'][0].keys()

dict_keys(['input_ids', 'labels'])

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
  output_dir="./bart-cnn-samsum-finetuned", # local directory
  hub_model_id="ObserverAI/bart-cnn-samsum-finetuned", # identifier on the Hub
  learning_rate=1e-5,
  num_train_epochs=1,
  weight_decay=0.01,
  auto_find_batch_size=True,
  evaluation_strategy='epoch',
  logging_steps=10
)

trainer = Trainer(
  model=model,
  tokenizer=tokenizer,
  args=training_args,
  train_dataset=tokenized_datasets['train'],
  eval_dataset=tokenized_datasets['validation']
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
trainer.push_to_hub()

In [ ]:
loaded_model = AutoModelForSeq2SeqLM.from_pretrained("ObserverAI/bart-cnn-samsum-finetuned")
testing_input = """
Alice: Hey Bob, did you finish the presentation for the client meeting tomorrow?
Bob: Not yet, Alice. I’m still working on the financial projections.
Alice: Oh no, we’re running out of time. Do you need any help with that?
Bob: Actually, yes. Can you handle the market analysis section?
Alice: Sure, I’ll get on it right away. By the way, have you confirmed the meeting time with the client?
Bob: Yes, it’s scheduled for 10 AM. I’ll send you the final agenda in a bit.
Alice: Great, thanks. Let’s make sure everything is ready tonight. We can review it together tomorrow morning.
Bob: Sounds like a plan. I appreciate your help.
Alice: No problem, we’re a team!
"""
output1 = generate_summary(testing_input, llm=loaded_model)
print("testing_input")
print ("------------------------")
print(testing_input)
print ("------------------------")
print("Model Generated Summary:")
print (output1)

testing_input
------------------------

Alice: Hey Bob, did you finish the presentation for the client meeting tomorrow?
Bob: Not yet, Alice. I’m still working on the financial projections.
Alice: Oh no, we’re running out of time. Do you need any help with that?
Bob: Actually, yes. Can you handle the market analysis section?
Alice: Sure, I’ll get on it right away. By the way, have you confirmed the meeting time with the client?
Bob: Yes, it’s scheduled for 10 AM. I’ll send you the final agenda in a bit.
Alice: Great, thanks. Let’s make sure everything is ready tonight. We can review it together tomorrow morning.
Bob: Sounds like a plan. I appreciate your help.
Alice: No problem, we’re a team!

------------------------
Model Generated Summary:
Bob is still working on the financial projections for the client meeting tomorrow. Alice is running out of time to finish the market analysis section of the presentation. Bob will send the final agenda in a bit.
